In [ ]:
using Revise

In [ ]:
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using RigidBodyDynamics: position_bounds, velocity_bounds, effort_bounds, Bounds
using StaticArrays
using RigidBodyTreeInspector
using LCPSim
using JuMP, Gurobi

In [ ]:
urdf_mech = parse_urdf(Float64, "box_valkyrie.urdf")
mechanism, base = planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)

basevis = Visualizer()[:boxval]
delete!(basevis)
vis = basevis[:robot]
setgeometry!(vis, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))

floor = planar_obstacle(default_frame(world), [0, 0, 1.], [0, 0, 0.], 1.)

contact_limbs = findbody.(mechanism, ["rh", "lh", "rf", "lf"])
hands = findbody.(mechanism, ["rh", "lh"])
feet = findbody.(mechanism, ["rf", "lf"])

env = Environment(
    Dict(vcat(
            [body => ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [floor])
                for body in feet],
            )));

function nominal_input(x0::MechanismState)
    u_nominal = clamp.(inverse_dynamics(x0, zeros(num_velocities(x0))), LCPSim.all_effort_bounds(x0.mechanism))
    feet = findbody.(x0.mechanism, ["rf", "lf"])
    weight = mass(x0.mechanism) * mechanism.gravitational_acceleration.v[3]
    u_nominal[parentindexes(velocity(x0, findjoint(x0.mechanism, "core_to_rf_z")))...] += weight / 2
    u_nominal[parentindexes(velocity(x0, findjoint(x0.mechanism, "core_to_lf_z")))...] += weight / 2
    u_nominal
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0.75])
set_configuration!(x0, findjoint(mechanism, "base_z"), [.9])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-.9])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-.9])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
u0 = nominal_input(x0)

contacts = [Point3D(default_frame(body), SVector(0., 0, 0)) for body in feet]
qq = [100, 0.1, 1, fill(0.1, num_positions(x0) - 3)...]
Q = diagm(vcat(qq, fill(0.1, num_velocities(x0))))
R = 0.001 * eye(num_velocities(x0))
K = LCPSim.ContactLQR.contact_lqr(x0, u0, Q, R, contacts)

controller = x -> begin
    -K * (state_vector(x) - vcat(q0, v0)) .+ u0
#     u0
end
Δt = 0.01

set_velocity!(x0, findjoint(mechanism, "base_x"), [-2])

results = LCPSim.simulate(x0, controller, env, Δt, 500, GurobiSolver(OutputFlag=0));


In [ ]:

set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
    for (body, contacts) in r.contacts
        for (i, contact) in enumerate(contacts)
            f = LCPSim.contact_force(contact)
            p = transform_to_root(x0, contact.point.frame) * contact.point
            v = vis[:forces][Symbol(body)][Symbol(i)]
            setgeometry!(v, DrakeVisualizer.PolyLine([p.v, (p + 0.1*f).v]; end_head=DrakeVisualizer.ArrowHead()))
        end
    end
end